# BookFindr
### Ignacio Díaz-Guardamino García

Esta aplicación escanea los libros de una biblioteca en .txt y recomienda, a partir de un libro, otro para leer a continuación.

In [1]:
import pandas as pd
import numpy as np
import gensim.models.word2vec as w2v
import multiprocessing
import glob, os
import re
import pprint
import sklearn.manifold
import matplotlib.pyplot as plt

In [2]:
text_corpus = []
booksdf = pd.DataFrame(columns = ['name', 'content']) 
for file in glob.glob("*.txt"):
    with open(file, 'r', encoding="utf8") as f:
        print(file)
        i=0
        cont = ""
        for line in f:
            l=re.sub("[^a-z\s]+", "", line.lower().rstrip("\n"))  #minusculas y espacios.
            words = l.split()
            text_corpus.append(words)
            cont=cont+" "+l
        df=pd.DataFrame([[file, cont]],columns = ['name', 'content']) 
        booksdf=pd.concat([df, booksdf])

(Trilogia El Siglo 02) El Invie - Ken Follett.txt
Duenas, Maria - El tiempo entre - Author.txt
El Abuelo Que Salto Por La Vent - Jonas Jonasson.txt
El aprendiz - Clive Cussler.txt
El laberinto azul - Douglas Preston.txt
Huesos en el jardin - Henning Mankell.txt
Inferno - Dan Brown.txt
La Cupula - Stephen King.txt
La De Los Tristes Destinos - Perez Galdos, Benito.txt
Los Dientes Del Tigre I - Clancy, Tom.txt
Los inocentes - David Baldacci.txt
Ola de calor NH1 - Richard Castle.txt
Pisando los talones - Henning Mankell.txt
Vargas Llosa, Mario - La tia Ju - Sin determinar.txt


In [3]:
booksdf.head()

,name,content
0,"Vargas Llosa, Mario - La tia Ju - Sin determin...",vargas llosa mario la tia julia y el escribi...
0,Pisando los talones - Henning Mankell.txt,la noche de san juan alguien agazapado tras u...
0,Ola de calor NH1 - Richard Castle.txt,richard castle nikki heat ola de calor ...
0,Los inocentes - David Baldacci.txt,los inocentes david baldacci traducci...
0,"Los Dientes Del Tigre I - Clancy, Tom.txt",los dientes del tigre i sobrecubierta n...


In [4]:
# Dimensionality of the resulting word vectors.
#more dimensions, more computationally expensive to train
#but also more accurate
#more dimensions = more generalized
num_features = 50
# Minimum word count threshold.
min_word_count = 1

# Number of threads to run in parallel.
#more workers, faster we train
num_workers = multiprocessing.cpu_count()

# Context window length.
context_size = 7


downsampling = 1e-1

# Seed for the RNG, to make the results reproducible.
#random number generator
#deterministic, good for debugging
seed = 1

songs2vec = w2v.Word2Vec(
    sg=1,
    seed=seed,
    workers=num_workers,
    size=num_features,
    min_count=min_word_count,
    window=context_size,
    sample=downsampling
)

songs2vec.build_vocab(text_corpus)
print (len(text_corpus))

166375


In [5]:
import time
start_time = time.time()

songs2vec.train(text_corpus, total_examples=songs2vec.corpus_count, epochs=10)

if not os.path.exists("trained"):
    os.makedirs("trained")

songs2vec.save(os.path.join("trained", "songs2vectors_ES.w2v"))

print("--- %s seconds ---" % (time.time() - start_time))

--- 31.8414945602417 seconds ---


In [6]:
songs2vec = w2v.Word2Vec.load(os.path.join("trained", "songs2vectors_ES.w2v"))

#### Let's explore our model

Find similar words

In [7]:
songs2vec.wv.most_similar("escalera")

[('compuerta', 0.841240406036377),
 ('rampa', 0.8333134651184082),
 ('caracol', 0.8266541361808777),
 ('reja', 0.8264082074165344),
 ('escalerilla', 0.8210001587867737),
 ('trasera', 0.8204702734947205),
 ('abertura', 0.8117176294326782),
 ('ranura', 0.8115399479866028),
 ('barandilla', 0.8112460374832153),
 ('nave', 0.8052106499671936)]

In [8]:
songs2vec.wv.most_similar("gris")

[('oscuro', 0.8849686980247498),
 ('terno', 0.8763387203216553),
 ('rojo', 0.8712367415428162),
 ('negro', 0.8699830770492554),
 ('blanco', 0.8614271283149719),
 ('color', 0.8585057854652405),
 ('canotier', 0.8457030057907104),
 ('blazer', 0.8449329137802124),
 ('ondulado', 0.8376966118812561),
 ('marrn', 0.8318960666656494)]

In [9]:
songs2vec.wv.most_similar("tiempo")

[('lapso', 0.7525243163108826),
 ('llevabais', 0.7447783946990967),
 ('prolongado', 0.7322405576705933),
 ('lote', 0.7293753623962402),
 ('pronstico', 0.7277312874794006),
 ('orlos', 0.7224562168121338),
 ('remedio', 0.7214159965515137),
 ('elucubrando', 0.7163774967193604),
 ('pretextos', 0.7095689177513123),
 ('plan', 0.7065356373786926)]

Words out of context

In [10]:
songs2vec.wv.doesnt_match("perro gato queso elefante".split())

D:\ProgramData\Anaconda3\lib\site-packages\gensim\models\keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'queso'

In [11]:
songs2vec.most_similar(positive=['amarillo', 'oscuro'], negative=['claro'])
#queen

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('rojo', 0.8155493140220642),
 ('gris', 0.7879650592803955),
 ('negro', 0.7748938798904419),
 ('amarillento', 0.7687134146690369),
 ('ladrillo', 0.7585065960884094),
 ('fieltro', 0.756888210773468),
 ('rectngulo', 0.7563862204551697),
 ('pulido', 0.754917562007904),
 ('revestido', 0.7504590749740601),
 ('tallado', 0.7446067333221436)]

Semantic distance between words

In [12]:
def nearest_similarity_cosmul(start1, end1, end2):
    similarities = songs2vec.wv.most_similar_cosmul(
        positive=[end2, start1],
        negative=[end1]
    )
    start2 = similarities[0][0]
    print("{0} es a {1}, lo que {2} es a {3}".format(start1, end1, start2, end2))

In [13]:
nearest_similarity_cosmul("beso", "amor", "tiempo")

beso es a amor, lo que rato es a tiempo


With the word vector embeddings in place, it is now time to calculate the normalised vector sum of each song. This process can take some time since it has to be done for each of 57,000 songs.

In [14]:

def songVector(row):
    vector_sum = 0
    words = row.lower().split()
    for word in words:
        vector_sum = vector_sum + songs2vec[word]
    vector_sum = vector_sum.reshape(1,-1)
    normalised_vector_sum = sklearn.preprocessing.normalize(vector_sum)
    return normalised_vector_sum


import time
start_time = time.time()

booksdf['bookV'] = booksdf['content'].apply(songVector)

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
D:\ProgramData\

**t-sne and random song selection** 

The songs have 50 dimensions each. Application of t-sne is memory intensive and hence it is slightly easier on the computer to use a random sample of the 57,000 songs.

In [15]:
book_vectors = []
from sklearn.model_selection import train_test_split

train, test = train_test_split(booksdf, test_size = 0.1)


for book_vector in train['bookV']:
    book_vectors.append(book_vector)

train.head(10)

,name,content,bookV
0,"Vargas Llosa, Mario - La tia Ju - Sin determin...",vargas llosa mario la tia julia y el escribi...,"[[0.14859587, -0.30749395, -0.065080516, 0.022..."
0,Huesos en el jardin - Henning Mankell.txt,en octubre de un kurt wallander cansado y re...,"[[0.16918841, -0.31160444, -0.07086668, 0.0194..."
0,Inferno - Dan Brown.txt,el smbolo perdido sobrecubierta none ...,"[[0.13659677, -0.30021873, -0.119918205, 0.011..."
0,Los inocentes - David Baldacci.txt,los inocentes david baldacci traducci...,"[[0.1624437, -0.32105866, -0.08532177, 0.01808..."
0,La Cupula - Stephen King.txt,la cpula la cpula stephen king la c...,"[[0.1397805, -0.314141, -0.065867186, 0.027044..."
0,Pisando los talones - Henning Mankell.txt,la noche de san juan alguien agazapado tras u...,"[[0.17335527, -0.32028005, -0.06865694, 0.0094..."
0,"Duenas, Maria - El tiempo entre - Author.txt",dueas mara el tiempo entre costuras r d...,"[[0.15067542, -0.3019987, -0.061541483, 0.0156..."
0,(Trilogia El Siglo 02) El Invie - Ken Follett.txt,sin determinar la cada de los gigantes po...,"[[0.1369806, -0.3400632, -0.069811985, 0.01079..."
0,El laberinto azul - Douglas Preston.txt,a las nueve y veinte de la noche llaman al ti...,"[[0.15810797, -0.2917135, -0.09894615, 0.01783..."
0,El aprendiz - Clive Cussler.txt,clive cussler y justin scott el apren...,"[[0.15788396, -0.30873355, -0.10232674, -0.003..."


I had a fairly measly 4gb machine and wasn't able to generate a more accurate model. However, one can play around with the number of iterations, learning rate and other factors to fit the model better. If you have too many dimensions (~300+), it might make sense to use PCA first and then t-sne.

In [16]:
X = np.array(book_vectors).reshape((12, 50))

start_time = time.time()
tsne = sklearn.manifold.TSNE(n_components=2, n_iter=250, random_state=0, verbose=2)

all_word_vectors_matrix_2d = tsne.fit_transform(X)

print("--- %s seconds ---" % (time.time() - start_time))

[t-SNE] Computing 11 nearest neighbors...
[t-SNE] Indexed 12 samples in 0.000s...
[t-SNE] Computed neighbors for 12 samples in 0.004s...
[t-SNE] Computed conditional probabilities for sample 12 / 12
[t-SNE] Mean sigma: 1125899906842624.000000
[t-SNE] Computed conditional probabilities in 0.050s
[t-SNE] Iteration 50: error = 54.1318932, gradient norm = 0.0973943 (50 iterations in 0.031s)
[t-SNE] Iteration 100: error = 68.5094910, gradient norm = 0.1255101 (50 iterations in 0.009s)
[t-SNE] Iteration 150: error = 62.3195572, gradient norm = 0.3673777 (50 iterations in 0.010s)
[t-SNE] Iteration 200: error = 47.8195572, gradient norm = 0.2617192 (50 iterations in 0.010s)
[t-SNE] Iteration 250: error = 51.5933495, gradient norm = 0.7344074 (50 iterations in 0.010s)
[t-SNE] KL divergence after 250 iterations with early exaggeration: 51.593349
[t-SNE] KL divergence after 251 iterations: 179769313486231570814527423731704356798070567525844996598917476803157260780028538760589558632766878171540458

In [17]:
df=pd.DataFrame(all_word_vectors_matrix_2d,columns=['X','Y'])

df.head(10)

train.head()

df.reset_index(drop=True, inplace=True)
train.reset_index(drop=True, inplace=True)

Joining two dataframes to obtain each song's corresponding X,Y co-ordinate.

In [18]:
two_dimensional_songs = pd.concat([train, df], axis=1)

two_dimensional_songs.head()

,name,content,bookV,X,Y
0,"Vargas Llosa, Mario - La tia Ju - Sin determin...",vargas llosa mario la tia julia y el escribi...,"[[0.14859587, -0.30749395, -0.065080516, 0.022...",18.393295,18.397182
1,Huesos en el jardin - Henning Mankell.txt,en octubre de un kurt wallander cansado y re...,"[[0.16918841, -0.31160444, -0.07086668, 0.0194...",-4.417850,-13.448872
2,Inferno - Dan Brown.txt,el smbolo perdido sobrecubierta none ...,"[[0.13659677, -0.30021873, -0.119918205, 0.011...",30.704376,11.756790
3,Los inocentes - David Baldacci.txt,los inocentes david baldacci traducci...,"[[0.1624437, -0.32105866, -0.08532177, 0.01808...",57.746433,11.980558
4,La Cupula - Stephen King.txt,la cpula la cpula stephen king la c...,"[[0.1397805, -0.314141, -0.065867186, 0.027044...",-29.095654,-4.028286


**Plotting the results**

Using plotly, I plotted the results so that it becomes easier to explore similar songs based on their colors and clusters.

In [19]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

import plotly.graph_objs as go

trace1 = go.Scatter(
    y = two_dimensional_songs['Y'],
    x = two_dimensional_songs['X'],
    text = two_dimensional_songs['name'],
    mode='markers',
    marker=dict(
        size= 16,#[10],#'7',
        color = np.random.randn(5717), #set color equal to a variable
        colorscale='Viridis',
        showscale=True
    )
)
data = [trace1]

iplot(data)

In [20]:
#esta funcion busca libros cercanos en el grafico.
def closest(df,book):
    x=df[df['name'] == book]["X"].values[0]
    y=df[df['name'] == book]["Y"].values[0]
    dmin=100000000
    name=""
    
    for index, b in df.iterrows():
        dist=(x - b["X"])**2+(y - b["Y"])**2   #distancia euclidea, no hace falta raiz para comparar
        if dist!=0 and dist<dmin:
            dmin=dist
            name=b["name"]
    return name

In [21]:
book="El aprendiz - Clive Cussler.txt"    
print("un libro similar a "+book+" en tu colección es "+closest(two_dimensional_songs,book))

un libro similar a El aprendiz - Clive Cussler.txt en tu colección es El laberinto azul - Douglas Preston.txt


In [23]:
book="Vargas Llosa, Mario - La tia Ju - Sin determinar.txt"    
print("un libro similar a "+book+" en tu colección es "+closest(two_dimensional_songs,book))

un libro similar a Vargas Llosa, Mario - La tia Ju - Sin determinar.txt en tu colección es El laberinto azul - Douglas Preston.txt
